In [1]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.regression.gmm import GMM

In [2]:
data = pd.read_excel('C:/Users/XiaoHuang/Desktop/Assignment2/Data.xlsx') #reading data 

In [3]:
r = np.array(data['r_t']) #generate monthly rate

In [4]:
print(r,type(r),len(r)) #check data

[0.03411 0.03259 0.03264 0.03523 0.03427 0.03574 0.03427 0.03884 0.03849
 0.03788 0.03859 0.03732 0.03783 0.03768 0.03783 0.03986 0.03849 0.03798
 0.04266 0.04531 0.04459 0.04418 0.04602 0.04459 0.0452  0.04546 0.04724
 0.05157 0.04775 0.04266 0.0447  0.04546 0.04485 0.04062 0.03453 0.03249
 0.03707 0.03808 0.03859 0.04154 0.04149 0.0419  0.04343 0.04807 0.04673
 0.04852 0.05488 0.05386 0.05285 0.05259 0.04938 0.05162 0.05411 0.04836
 0.06099 0.05999 0.05641 0.05361 0.06125 0.05845 0.06481 0.06915 0.0666
 0.06945 0.06685 0.06447 0.06787 0.07728 0.06303 0.06415 0.06512 0.06309
 0.0593  0.05895 0.06109 0.05819 0.05274 0.04449 0.04439 0.04062 0.03229
 0.03406 0.03676 0.0387  0.05019 0.0506  0.04408 0.04531 0.03992 0.03828
 0.03351 0.03127 0.03147 0.03483 0.03249 0.03401 0.03625 0.03513 0.04307
 0.04429 0.04541 0.04917 0.04867 0.05571 0.05579 0.05875 0.05987 0.06527
 0.07436 0.08312 0.08581 0.07335 0.06928 0.07447 0.07364 0.0755  0.07377
 0.085   0.08771 0.07834 0.07681 0.07715 0.09215 0.0

In [5]:
diff_r = np.diff(r) #generate dr

In [6]:
print(diff_r,type(diff_r),len(diff_r))

[-1.520e-03  5.000e-05  2.590e-03 -9.600e-04  1.470e-03 -1.470e-03
  4.570e-03 -3.500e-04 -6.100e-04  7.100e-04 -1.270e-03  5.100e-04
 -1.500e-04  1.500e-04  2.030e-03 -1.370e-03 -5.100e-04  4.680e-03
  2.650e-03 -7.200e-04 -4.100e-04  1.840e-03 -1.430e-03  6.100e-04
  2.600e-04  1.780e-03  4.330e-03 -3.820e-03 -5.090e-03  2.040e-03
  7.600e-04 -6.100e-04 -4.230e-03 -6.090e-03 -2.040e-03  4.580e-03
  1.010e-03  5.100e-04  2.950e-03 -5.000e-05  4.100e-04  1.530e-03
  4.640e-03 -1.340e-03  1.790e-03  6.360e-03 -1.020e-03 -1.010e-03
 -2.600e-04 -3.210e-03  2.240e-03  2.490e-03 -5.750e-03  1.263e-02
 -1.000e-03 -3.580e-03 -2.800e-03  7.640e-03 -2.800e-03  6.360e-03
  4.340e-03 -2.550e-03  2.850e-03 -2.600e-03 -2.380e-03  3.400e-03
  9.410e-03 -1.425e-02  1.120e-03  9.700e-04 -2.030e-03 -3.790e-03
 -3.500e-04  2.140e-03 -2.900e-03 -5.450e-03 -8.250e-03 -1.000e-04
 -3.770e-03 -8.330e-03  1.770e-03  2.700e-03  1.940e-03  1.149e-02
  4.100e-04 -6.520e-03  1.230e-03 -5.390e-03 -1.640e-03 -4.770

In [7]:
r = np.delete(r, len(r)-1, axis=0)

**fitting unrestricted** 

In [63]:
# 4  moment conditions for unrestricted 
class gmm(GMM):
    def momcond(self, params):
        alpha, beta, sigma2, gama = params
        endog = self.endog
        exog = self.exog.squeeze()

        mom_cond1 = endog - (alpha + beta * exog)/12
        mom_cond2 = (endog - (alpha + beta * exog)) * r/12
        mom_cond3 = (endog - alpha - beta * exog) ** 2 - sigma2 * (exog ** (2 * gama)) / 12
        mom_cond4 = ((endog - alpha - beta * exog) ** 2 - sigma2 * (exog ** (2 * gama)) / 12) * r
        e = np.column_stack((mom_cond1, mom_cond2, mom_cond3, mom_cond4))
        return e

In [22]:
#fitting 
param0 = np.array([1.0, 1.0, 1.0, 1.0])
res = gmm(endog=diff_r, exog=r, instrument=r, k_moms=4, k_params=4).fit(param0)
res.model.exog_names[:] = 'alpha beta sigma2 gama'.split()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 6
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 66
         Function evaluations: 79
         Gradient evaluations: 79
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
                                 gmm Results                                  
Dep. Variable:                      y   Hansen J:                    5.396e-09
Model:                            gmm   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:

**fitting CIR SR with gama as 0.5**

In [16]:
# 3 conditions for estimating CIR SR.
class gmm(GMM):
    def momcond(self, params):
        sigma2 = params
        alpha,beta = 0,0 
        gama = 1
        endog = self.endog
        exog = self.exog.squeeze()

        mom_cond1 = endog - (alpha + beta * exog)/12
        mom_cond2 = (endog - (alpha + beta * exog)) * r/12
        mom_cond3 = (endog - alpha - beta * exog) ** 2 - sigma2 * (exog ** (2 * 0.5)) / 12
        mom_cond4 = ((endog - alpha - beta * exog) ** 2 - sigma2 * (exog ** (2 * 0.5)) / 12) * r
        e = np.column_stack((mom_cond1, mom_cond2, mom_cond3, mom_cond4))
        return e

In [17]:
# fitting 
param0 = np.array([1.0])
res = gmm(endog=diff_r, exog=r, instrument=r, k_moms=4, k_params=1).fit(param0)
res.model.exog_names[:] = 'sigma2'.split()
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 2
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.024864
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.024457
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.024459
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3


LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [64]:
dic = {'Unrestricted': [0.01, 0.01, 0.01, 0.1],
       'Merton': [0.01, np.nan, 0.01, np.nan],
       'Vasicek': [0.01, 0.01, 0.01, np.nan],
       'CIR SR': [0.01, 0.01, 0.01, np.nan],
       'Dothan': [np.nan, np.nan, 0.01, np.nan],
       'GBM':    [np.nan, 0.01, 0.01, np.nan],
       'BS':     [0.01, 0.01, 0.01, np.nan],
       'CIR VR': [np.nan, np.nan, 1, np.nan],
       'CEV':    [np.nan, 0.01, 0.1, 1],
       }


In [65]:
df= pd.DataFrame([dic])

In [67]:
new_df = df.T

In [68]:
for model in new_df.index:
    temp = new_df.loc[model].values[0:]
    init_param = temp[pd.notna(temp)]
    Gmm = gmm(model=model, endog=diff_r, exog=r, instrument=None,
              k_moms=4, k_params=len(init_param))
    res = Gmm.fit(init_param, maxiter=2)

ValueError: setting an array element with a sequence.